In [9]:
import pandas as pd
import os
from datetime import datetime
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
listaParlamentar = pd.read_csv("./dados/ListaParlamentarEmExercicio.csv", sep=";" )
listaParlamentar = listaParlamentar.groupby(['CodigoParlamentar']).first().reset_index()

In [4]:
#decorator
def tempo_de_execucao(func):
    def wrapper(*args, **kwargs):
        inicio = time.time()
        resultado = func(*args, **kwargs)
        fim = time.time()
        duracao = fim - inicio
        print(f"A função {func.__name__} levou {duracao:.4f} segundos para ser executada.")
        return resultado
    return wrapper

In [15]:
@tempo_de_execucao
def CriarDF(pasta_csv):
    arquivos = [f for f in os.listdir(pasta_csv) if f.endswith('.csv')]
    dataframes = []
    for arquivo in arquivos:
        caminho_completo = os.path.join(pasta_csv, arquivo)
        df = pd.read_csv(caminho_completo)
        dataframes.append(df)
    df_final = pd.concat(dataframes, ignore_index=True)
    return df_final
    

def formatar_valores_monetarios(df):
    for coluna in df.columns:
        if df[coluna].dtype == 'object':
            df[coluna] = df[coluna].str.replace('.', '').str.replace(',', '.').astype(float)
    return df

@tempo_de_execucao
def Gastos():
    gastos = CriarDF('./dados/gastos')
    gastosF = gastos.drop(["Passagens","Total"], axis='columns')
    gastosF = formatar_valores_monetarios(gastosF)
    gastosF = gastosF.fillna(0)
    gastosF = gastosF[gastosF["ano"]>2022].groupby("CodigoParlamentar").sum().reset_index()
    gastosF = gastosF.drop(["ano"], axis='columns')
    gastosF["total"] = gastosF.sum(axis=1)
    gastosF["GastouMedia"] = gastosF['total'] > (gastosF[["CodigoParlamentar","total"]].sum()["total"]/81)
    return gastosF[["CodigoParlamentar","GastouMedia"]]

@tempo_de_execucao
def Autorias():
    autorias = CriarDF('./dados/autorias')
    autorias = autorias[autorias["AnoMateria"]>2022]
    autorias["IndicadorAutorPrincipal"] = autorias["IndicadorAutorPrincipal"].replace({"Sim": 1, "Não": 0.5})
    autorias = autorias.groupby(["codigoParlamentar"]).sum().reset_index()[["codigoParlamentar","IndicadorAutorPrincipal"]]
    autorias = autorias.rename(columns={"codigoParlamentar":"CodigoParlamentar","IndicadorAutorPrincipal":"NotaAutorias"})
    return autorias

@tempo_de_execucao
def Discursos():
    discursos = CriarDF('./dados/discursos')
    discursos = discursos.groupby("codigoParlamentar").count().reset_index()[["codigoParlamentar","DataPronunciamento"]].rename(columns={"codigoParlamentar":"CodigoParlamentar","DataPronunciamento":"notaPronunciamentos"})
    discursos["pronunciamentosMedia"] = discursos["notaPronunciamentos"] > (discursos["notaPronunciamentos"].sum()/81)
    return discursos

@tempo_de_execucao
def Relatorias():
    relatorias = CriarDF('./dados/relatorias')
    relatoriasF = relatorias[relatorias["AnoMateria"]>2022].groupby("codigoParlamentar").count().reset_index()[["codigoParlamentar","Identificacao"]].rename(columns={"codigoParlamentar":"CodigoParlamentar","Identificacao":"Relatorias"})
    relatoriasF["relatoriasMedia"] = relatoriasF["Relatorias"] > (relatoriasF["Relatorias"].sum()/81)
    return relatoriasF

@tempo_de_execucao
def Comissoes():
    comissoes = CriarDF('./dados/comissoes')
    comissoes["DescricaoParticipacao"] = comissoes["DescricaoParticipacao"].replace({"Titular": 1, "Suplente": 0.5})
    comissoes["DescricaoParticipacao"] = pd.to_numeric(comissoes["DescricaoParticipacao"], errors='coerce')
    comissoes = comissoes[["codigoParlamentar","DescricaoParticipacao"]].groupby("codigoParlamentar").sum().reset_index().rename(columns={"codigoParlamentar":"CodigoParlamentar","DescricaoParticipacao":"NotaComissoes"})
    return comissoes

def verificarVoto(df):
    subtrai2 = ["Abstenção", "Não Compareceu", "Presente – Não registrou voto"]
    soma2 = ["Presidente (art. 51 RISF)"]
    soma1 = ["Não", "Sim", "Votou"]
    valor = 0
    if df["DescricaoVoto"] in subtrai2:
        valor = -2
    elif df["DescricaoVoto"] in soma2:
        valor = 2
    elif df["DescricaoVoto"] in soma1:
        valor = 1
    return valor

@tempo_de_execucao
def Votacoes():
    votacoes = CriarDF('./dados/votacoes')
    votacoes['DataSessaoPlenaria'] = pd.to_datetime(votacoes['DataSessaoPlenaria'])
    votacoes = votacoes[votacoes["DataSessaoPlenaria"] > pd.to_datetime('2023-01-01')] 
    tabela = []
    for i,r in votacoes.iterrows():
        nota = verificarVoto(r)
        tabela.append([nota,r["codigoParlamentar"],r["DescricaoVoto"]])
    df = pd.DataFrame(tabela,columns=["NotaVotacoes","CodigoParlamentar","voto"])
    return df.groupby(["CodigoParlamentar"]).sum().reset_index()[["CodigoParlamentar","NotaVotacoes"]]

@tempo_de_execucao
def Ranking():
    gastos = Gastos()
    autorias = Autorias()
    discursos = Discursos()
    relatorias = Relatorias()
    comissoes = Comissoes()
    votacoes = Votacoes()
    ranking = gastos.merge(autorias[["CodigoParlamentar","NotaAutorias"]],how='left', on='CodigoParlamentar')
    ranking = ranking.merge(relatorias[["CodigoParlamentar","Relatorias"]],how='left', on='CodigoParlamentar')
    ranking = ranking.merge(comissoes[["CodigoParlamentar","NotaComissoes"]],how='left', on='CodigoParlamentar')
    ranking = ranking.merge(votacoes[["CodigoParlamentar","NotaVotacoes"]],how='left', on='CodigoParlamentar')
    ranking = ranking.fillna(0)
    ranking["Total"] = ranking["NotaAutorias"] + ranking["Relatorias"] + ranking["NotaComissoes"] + ranking["NotaVotacoes"]
    return ranking    

In [16]:
ranking = Ranking()
#ranking.to_csv('./dados/ranking.csv',index=False)
ranking

A função CriarDF levou 0.2060 segundos para ser executada.
A função Gastos levou 0.2343 segundos para ser executada.
A função CriarDF levou 0.5757 segundos para ser executada.
A função Autorias levou 0.6728 segundos para ser executada.
A função CriarDF levou 0.2000 segundos para ser executada.
A função Discursos levou 0.2060 segundos para ser executada.
A função CriarDF levou 0.3740 segundos para ser executada.
A função Relatorias levou 0.3820 segundos para ser executada.
A função CriarDF levou 0.2120 segundos para ser executada.
A função Comissoes levou 0.2260 segundos para ser executada.
A função CriarDF levou 0.9947 segundos para ser executada.
A função Votacoes levou 2.1109 segundos para ser executada.
A função Ranking levou 3.8420 segundos para ser executada.


,CodigoParlamentar,GastouMedia,NotaAutorias,Relatorias,NotaComissoes,NotaVotacoes,Total
0,22,True,207.5,47,104.0,151,509.5
1,35,False,57.0,3,55.5,-9,106.5
2,70,False,57.0,12,111.5,-30,150.5
3,285,True,135.5,13,44.0,29,221.5
4,345,False,199.0,28,110.5,140,477.5
...,...,...,...,...,...,...,...
76,6358,False,50.5,30,15.5,58,154.0
77,6363,False,6.0,5,5.5,16,32.5
78,6366,False,2.5,4,11.5,11,29.0
79,6369,True,56.5,25,13.0,139,233.5
